### A. Upload CSV to S3 (using boto3)

In [ ]:
# install dependencies or put them in a requirements.txt file
# !pip install pandas
#!pip install boto3


In [29]:
# Import dependencies
import boto3
import os

### LOAD DATA INTO AWS

In [ ]:
# Setup
bucket_name = "med-etl-bucket"
folder_data = "./medoptix_data"
target_folder = "/medoptix/raw/"

# AWS credentials assumed to be in ~/.aws/credentials or env vars
s3 = boto3.client('s3')

# Upload all CSVs in folder with ACL: bucket-owner-full-control
try:
    for filename in os.listdir(folder_data):
        if filename.endswith(".csv"):
            filepath = os.path.join(folder_data, filename)
            s3.upload_file(
                Filename=filepath,
                Bucket=bucket_name,
                Key= target_folder + filename
            )
            print(f"Uploaded {filename} successfully")
except Exception as e:
    print(f"❌ Upload failed: {e}")

Uploaded clinics.csv successfully
Uploaded dropout_flags.csv successfully
Uploaded feedback.csv successfully
Uploaded interventions.csv successfully
Uploaded patients.csv successfully
Uploaded sessions.csv successfully


### B. LOAD DATA FROM AWS S3 --------- POSTGRESQL

In [31]:


s3 = boto3.client('s3')

bucket = "med-etl-bucket"
prefix = "/medoptix/raw/"

files  = ["patients.csv", "sessions.csv"]

for file in files:
    s3.download_file(bucket, prefix + file, file)
    print(f"Downloaded {file} successfully")

Downloaded patients.csv successfully
Downloaded sessions.csv successfully


### C. Loading Data from AWS S3 To Posgres Database

In [3]:
import os
import pandas as pd
from sqlalchemy import create_engine

In [ ]:
# # Upload all CSVs in folder with ACL: bucket-owner-full-control

def get_db_engine():
    db_url = (
        f"postgresql://{"avnadmin"}:{"AVNS_WUAxDaCB98_BJRKU3C7"}@"
        f"{"myproject-db-ooluwakemmie-a56d.j.aivencloud.com"}:{"17947"}/{"defaultdb"}?"
    )

    return create_engine(db_url)

In [ ]:
def upload_data():
    patients = pd.read_csv("C:/Users/ooluw/Downloads/MedOptix/med_env/medoptix_data/patients.csv")
    sessions = pd.read_csv("C:/Users/ooluw/Downloads/MedOptix/med_env/medoptix_data/sessions.csv")
    feedback = pd.read_csv("C:/Users/ooluw/Downloads/MedOptix/med_env/medoptix_data/feedback.csv")
    interventions = pd.read_csv("C:/Users/ooluw/Downloads/MedOptix/med_env/medoptix_data/interventions.csv")
    clinics = pd.read_csv("C:/Users/ooluw/Downloads/MedOptix/med_env/medoptix_data/clinics.csv")    
    dropout_flags = pd.read_csv("C:/Users/ooluw/Downloads/MedOptix/med_env/medoptix_data/dropout_flags.csv")
    
    # Step 2: Create database engine
    engine = get_db_engine()

    # Step 3: Upload in referential order with error handling
    with engine.begin() as connection:  # Automatically handles transactions
 
        # Upload patients table
        patients.to_sql(
            "patients",
            connection, 
            if_exists='append', 
            index=False,
            chunksize=1000,
            method='multi'
        )

        # Upload sessions table
        sessions.to_sql(
            "sessions",
            connection, 
            if_exists='append', 
            index=False,
            chunksize=1000,
            method='multi'
        )

        # Upload feedback table
        feedback.to_sql(
            "feedback",
            connection,
            if_exists='append',
            index=False,    
            chunksize=1000,
            method='multi'  
        )

        # Upload interventions tables
        interventions.to_sql(
            "interventions",
            connection,
            if_exists='append',
            index=False,
            chunksize=1000,
            method='multi'
        )

        # Upload clinics table
        clinics.to_sql(
            "clinics",
            connection,
            if_exists='append',
            index=False,
            chunksize=1000,
            method='multi'
        )

        # Upload dropout flags table
        dropout_flags.to_sql(
            "dropout_flags",
            connection,
            if_exists='append',
            index=False,
            chunksize=1000,
            method='multi'
        )

    print("Data uploaded successfully to the database.")


upload_data()

Data uploaded successfully to the database.
